In [1]:
# いつも使うやつ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# pandasで全ての列を表示
pd.options.display.max_columns = None

# 図をipython notebook内で表示
%matplotlib inline

# DeplicatedWarningを避けるため
import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning)

import sys
sys.path.append('./lib/')

# 交差検定をサクッとやるためのモジュールを読み込む
from sklearn.cross_validation import cross_val_score

# 説明変数つくるための関数といろいろ
import feature_process_helper
import get_data

# submissionを書き込む関数
import write_submission

/Users/k.tamashiro/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


ここのRの内容が気になるので、Pythonに書き換える

https://www.kaggle.com/jasonm/titanic/large-families-not-good-for-survival/code

In [2]:
# loaddata <- function(file) {
#  data <- read.csv(file, header = TRUE, stringsAsFactors=F)
#  # compute family size on dataset (including self)
#  data$FamilySize <- data$SibSp + data$Parch + 1 
#  data
# }

In [2]:
def load_data(file):
    df = pd.read_csv(file)
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    return df

train_df = load_data('./data/train.csv')
test_df = load_data('./data/test.csv')

In [6]:
# change survived from integer to boolean

# data$Survived <- as.logical(data$Survived)
# levels(data$Survived) <- c("Not survived", "Survived")

# これは特に大丈夫っぽい

In [8]:
# make explicit factor levels for specific variables: 3=Pclass, 5=Sex, 12=Embarked
# for(i in c(3,5,12)) {
#  data[,i] <- as.factor(data[,i])
#}

# こちらはPclassとSexとEmbarkedをfactorに変更している
# あとでやる

In [9]:
# cabins <- data$Cabin # 1309 rows
# n_occur <- data.frame(table(Var1=cabins))	# 187 rows
# remove missing cabin and/or juse the cabin letter code (e.g. D)
# n_occur <- subset(n_occur, nchar(as.character(Var1)) > 1) # 183 rows

# sharedCabins <- n_occur$Var1[n_occur$Freq > 1]
# data$FamilySizeAdj <- data$FamilySize
# print(table(data$FamilySize))

# sharedInd <- data$FamilySizeAdj == 1 & data$Cabin %in% sharedCabins
# data$FamilySizeAdj[sharedInd] <- 2
# rowCount <- sum(sharedInd)
# print(c("adjusted rows", rowCount)) # 27 rows
# print(table(data$FamilySizeAdj))

In [103]:
contact_df = pd.concat([train_df, test_df], axis=0)

In [104]:
cabins = contact_df[ 'Cabin']

In [105]:
len(cabins)

1309

In [108]:
len(cabins.value_counts())

186

In [111]:
cabins_df = pd.DataFrame(cabins.value_counts(), columns=['Cabin']).reset_index().rename(columns={'CabinCount':'CabinName', 'Cabin': 'CabinCount'})

In [112]:
cabins_df.head()

,index,CabinCount
0,C23 C25 C27,6
1,B57 B59 B63 B66,5
2,G6,5
3,F33,4
4,C22 C26,4


In [113]:
# n_occur <- subset(n_occur, nchar(as.character(Var1)) > 1) # 183 rows
# １以上の値を探しています

one_lt_cabins = cabins_df[cabins_df['CabinCount'] > 1]

In [114]:
one_lt_cabins.head()

,index,CabinCount
0,C23 C25 C27,6
1,B57 B59 B63 B66,5
2,G6,5
3,F33,4
4,C22 C26,4


In [115]:
len(one_lt_cabins)

79

諦める

In [2]:
# データをそれぞれ読み込む
train_df, test_df, submission_df = get_data.get(path='./data')

In [4]:
train = train_df.copy()
test = test_df.copy()

In [ ]:
# 相関みてみるー

In [5]:
train, test = feature_process_helper.names(train, test)
train, test = feature_process_helper.age_impute(train, test)
train, test = feature_process_helper.cabin_num(train, test)
train, test = feature_process_helper.cabin(train, test)
train, test = feature_process_helper.embarked_impute(train, test)
train, test = feature_process_helper.fam_size(train, test)
train['Ticket_Len'] = train['Ticket'].apply(lambda x: len(x))
test['Ticket_Len'] = test['Ticket'].apply(lambda x: len(x))
train, test = feature_process_helper.dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked',
                                                                     'Cabin_Letter', 'Name_Title', 'Fam_Size'])
train, test = feature_process_helper.drop(train, test, bye = ['Ticket'])

In [8]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(criterion='gini', 
                             n_estimators=50,
                             min_samples_split=16,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
# rf.fit(train.iloc[:, 2:], train.iloc[:, 1])
# print("%.4f" % rf.oob_score_)

コメントアウト

In [6]:
# k分割交差検証
from sklearn.cross_validation import StratifiedKFold
kfold = StratifiedKFold(y = train.iloc[:, 1],
                       n_folds = 10,
                       random_state = 1)

scores = []

for k, (_train, _test) in enumerate(kfold):
    rf.fit(train.iloc[:, 2:], train.iloc[:, 1])
    score = rf.score(train.iloc[:, 2:].ix[_test], train.iloc[:, 1].ix[_test])
    scores.append(score)
    print('Fold: %s, Class dist.: %s, Acc: %.3f' % 
         (k + 1, np.bincount(train.iloc[:, 1][_train]), score))
# 正解率の平均と標準偏差を出力
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

Fold: 1, Class dist.: [494 307], Acc: 0.889
Fold: 2, Class dist.: [494 307], Acc: 0.922
Fold: 3, Class dist.: [494 308], Acc: 0.809
Fold: 4, Class dist.: [494 308], Acc: 0.910
Fold: 5, Class dist.: [494 308], Acc: 0.944
Fold: 6, Class dist.: [494 308], Acc: 0.876
Fold: 7, Class dist.: [494 308], Acc: 0.854
Fold: 8, Class dist.: [494 308], Acc: 0.831
Fold: 9, Class dist.: [494 308], Acc: 0.921
Fold: 10, Class dist.: [495 308], Acc: 0.886
CV accuracy: 0.884 +/- 0.041


In [7]:
# scikit-learnでは実装されているんだって
# 交差検定をサクッとやるためのモジュールを読み込む
from sklearn.cross_validation import cross_val_score
warnings.filterwarnings("ignore", category=DeprecationWarning)
cv_scores = cross_val_score(rf, train.iloc[:, 2:], train.iloc[:, 1], cv=10, n_jobs=-1)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy: 0.884 +/- 0.041


うーん。結構離れている気がする

In [8]:
write_submission._write(model=rf, train=train, test=test, submission_df=submission_df)

## アンサンブルしてみる

In [6]:
rf_param = {
    "n_estimators": [5, 10, 50, 100, 300],
    "max_features": [3, 5, 10, 15, 20],
    "min_samples_split": [3, 5, 10, 20],
    "max_depth": [3, 5, 10, 20]
}
lr_param = {
    "C": list(np.logspace(0, 4, 10))
}

In [9]:
from sklearn.linear_model import LogisticRegression
rf = RandomForestClassifier()
lr = LogisticRegression()

In [10]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import  RandomizedSearchCV

In [12]:
params = {}
params.update({"rf__" + k: v for k, v in rf_param.items()})
params.update({"lr__" + k: v for k, v in lr_param.items()})

eclf = VotingClassifier(estimators=[("rf", rf), ("lr", lr)],voting="soft")

clf = RandomizedSearchCV(eclf,
                         param_distributions=params,
                         cv=5,
                         n_iter=100,
                         n_jobs=1,
                         verbose=2)

In [13]:
clf.fit(train.iloc[:, 2:], train.iloc[:, 1])

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] rf__n_estimators=5, rf__min_samples_split=20, rf__max_features=3, rf__max_depth=20, lr__C=3593.8136638 
[CV]  rf__n_estimators=5, rf__min_samples_split=20, rf__max_features=3, rf__max_depth=20, lr__C=3593.8136638, total=   0.1s
[CV] rf__n_estimators=5, rf__min_samples_split=20, rf__max_features=3, rf__max_depth=20, lr__C=3593.8136638 
[CV]  rf__n_estimators=5, rf__min_samples_split=20, rf__max_features=3, rf__max_depth=20, lr__C=3593.8136638, total=   0.0s
[CV] rf__n_estimators=5, rf__min_samples_split=20, rf__max_features=3, rf__max_depth=20, lr__C=3593.8136638 
[CV]  rf__n_estimators=5, rf__min_samples_split=20, rf__max_features=3, rf__max_depth=20, lr__C=3593.8136638, total=   0.0s
[CV] rf__n_estimators=5, rf__min_samples_split=20, rf__max_features=3, rf__max_depth=20, lr__C=3593.8136638 
[CV]  rf__n_estimators=5, rf__min_samples_split=20, rf__max_features=3, rf__max_depth=20, lr__C=3593.8136638, total=   0.0s
[CV] 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  rf__n_estimators=5, rf__min_samples_split=20, rf__max_features=3, rf__max_depth=20, lr__C=3593.8136638, total=   0.0s
[CV] rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=20, rf__max_depth=10, lr__C=21.5443469003 
[CV]  rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=20, rf__max_depth=10, lr__C=21.5443469003, total=   0.1s
[CV] rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=20, rf__max_depth=10, lr__C=21.5443469003 
[CV]  rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=20, rf__max_depth=10, lr__C=21.5443469003, total=   0.1s
[CV] rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=20, rf__max_depth=10, lr__C=21.5443469003 
[CV]  rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=20, rf__max_depth=10, lr__C=21.5443469003, total=   0.1s
[CV] rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=20, rf__max_depth=10, lr__C=21.5443469003 
[CV]  rf__n_estimators=50, rf__min_samples_spli

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  2.5min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=VotingClassifier(estimators=[('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))],
         n_jobs=1, voting='soft', weights=None),
          fit_params={}, iid=True, n_iter=100, n_jobs=1,
          param_distributions={'rf__n_estimators': [5, 10, 50, 100, 300], 'rf__max_features': [3, 5, 10, 15, 20], 'rf__min_samples_split': [3, 5, 10, 20], 'rf__max_depth': [3, 5, 10, 20], 'lr__C': [1.0, 2.7825594022071245, 7.7426368268112693, 21.544346900318832, 59.948425031894089, 166.81005372000593, 464.15888336127773, 1291.5496650148827, 3593.8136638046258, 10000.0]},
          pre_dispatch='2*n_jobs

In [15]:
clf.best_params_

{'lr__C': 7.7426368268112693,
 'rf__max_depth': 20,
 'rf__max_features': 20,
 'rf__min_samples_split': 5,
 'rf__n_estimators': 5}

In [14]:
clf.best_estimator_

VotingClassifier(estimators=[('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features=20, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=5, min_weight_fraction_leaf=0.0,
            n_estima...ty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))],
         n_jobs=1, voting='soft', weights=None)

In [22]:
print(clf.predict(test.iloc[:, 1:]))

[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 1
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0
 0 1 1 1 1 1 0 1 0 0 1]


In [23]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
cv_scores = cross_val_score(clf, train.iloc[:, 2:], train.iloc[:, 1], cv=10, n_jobs=-1)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=15, rf__max_depth=20, lr__C=2.78255940221 
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=15, rf__max_depth=20, lr__C=2.78255940221 
[CV] rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=15, rf__max_depth=20, lr__C=2.78255940221 
[CV] rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=15, rf__max_depth=20, lr__C=2.78255940221 
[CV]  rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=15, rf__max_depth=20, lr__C=2.78255940221, total=   0.3s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=15, rf__max_depth=20, lr__C=2.78255940221 
[CV]  rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=15, rf__max_depth=20, lr__C=2.78255940221, total=   0.3s
[CV]  rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=15, rf__max_depth=20, lr__C=2.78255940221, total=   0.3s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV] rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=15, rf__max_depth=20, lr__C=2.78255940221 
[CV] rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=15, rf__max_depth=20, lr__C=2.78255940221 
[CV]  rf__n_estimators=50, rf__min_samples_split=10, rf__max_features=15, rf__max_depth=20, lr__C=2.78255940221, total=   0.3s


...

[CV]  rf__n_estimators=300, rf__min_samples_split=20, rf__max_features=3, rf__max_depth=3, lr__C=3593.8136638, total=   1.0s
[CV] rf__n_estimators=300, rf__min_samples_split=20, rf__max_features=3, rf__max_depth=3, lr__C=3593.8136638 
[CV]  rf__n_estimators=100, rf__min_samples_split=5, rf__max_features=10, rf__max_depth=20, lr__C=10000.0, total=   0.3s
[CV] rf__n_estimators=100, rf__min_samples_split=5, rf__max_features=10, rf__max_depth=20, lr__C=10000.0 
[CV]  rf__n_estimators=100, rf__min_samples_split=5, rf__max_features=10, rf__max_depth=20, lr__C=10000.0, total=   0.3s
[CV] rf__n_estimators=100, rf__min_samples_split=5, rf__max_features=10, rf__max_depth=20, lr__C=10000.0 
[CV]  rf__n_estimators=300, rf__min_samples_split=20, rf__max_features=3, rf__max_depth=3, lr__C=3593.8136638, total=   0.6s
[CV] rf__n_estimators=300, rf__min_samples_split=20, rf__max_features=3, rf__max_depth=3, lr__C=3593.8136638 
[CV]  rf__n_estimators=100, rf__min_samples_split=5, rf__max_features=10, rf

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  2.6min finished


[CV]  rf__n_estimators=300, rf__min_samples_split=5, rf__max_features=10, rf__max_depth=10, lr__C=1.0, total=   1.1s
[CV] rf__n_estimators=300, rf__min_samples_split=5, rf__max_features=10, rf__max_depth=10, lr__C=1.0 
[CV]  rf__n_estimators=300, rf__min_samples_split=5, rf__max_features=10, rf__max_depth=10, lr__C=1.0, total=   0.8s
[CV] rf__n_estimators=300, rf__min_samples_split=5, rf__max_features=10, rf__max_depth=10, lr__C=1.0 
[CV]  rf__n_estimators=300, rf__min_samples_split=5, rf__max_features=10, rf__max_depth=10, lr__C=1.0, total=   0.6s
[CV] rf__n_estimators=300, rf__min_samples_split=5, rf__max_features=10, rf__max_depth=10, lr__C=1.0 
[CV]  rf__n_estimators=300, rf__min_samples_split=5, rf__max_features=10, rf__max_depth=10, lr__C=1.0, total=   0.6s
[CV] rf__n_estimators=300, rf__min_samples_split=5, rf__max_features=10, rf__max_depth=10, lr__C=1.0 
[CV]  rf__n_estimators=300, rf__min_samples_split=5, rf__max_features=10, rf__max_depth=10, lr__C=1.0, total=   0.6s
[CV] rf

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  2.4min finished


CV accuracy: 0.884 +/- 0.041


In [24]:
write_submission._write(model=clf, train=train, test=test, submission_df=submission_df)

すごい時間かかった割に、0.78947  
orz